# Transfer Learning

In [1]:
import torch
torch.__version__

'1.7.1'

## Set device

In [41]:
# available_device = "cuda" if torch.cuda.is_available() else "cpu"
available_device = "cpu"

print(f"{available_device} is available")
device = torch.device(device=available_device)
device

cpu is available


device(type='cpu')

## Hyperparameters

In [42]:
in_channel = 3
num_classes = 10
lr = 0.001
batch_size = 1024
num_epochs = 5

## Load pretrained model and modify it

In [43]:
import sys
import torchvision

In [64]:
model = torchvision.models.vgg16(pretrained=True)
print(model)

# sys.exit()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [65]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

#### Pretrain all layers

In [56]:
model.avgpool = Identity()

# modules = []
# modules.append(nn.Linear(10, 10))
# modules.append(nn.Linear(10, 10))

# sequential = nn.Sequential(*modules)

model.classifier = nn.Linear(512,10)

# model.classifier.add_module("7",nn.Linear(512,10))

# model.classifier.append()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#### Pretrain only last layers

In [66]:
for param in model.parameters():
    param.requires_grad = False
    
model.avgpool = Identity()

sequential_layer = nn.Sequential(nn.Linear(512, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, 10))

model.classifier = sequential_layer


print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Load dataset

In [47]:
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [48]:
train_set = datasets.CIFAR10(root="pytorch_dataset/",
                     transform=transforms.ToTensor(),
                     download=True
                    )

test_set = datasets.CIFAR10(root="pytorch_dataset/",
                     transform=transforms.ToTensor(),
                    train=False,
                     download=True
                    )

Files already downloaded and verified
Files already downloaded and verified


In [49]:
train_loader = DataLoader(dataset=train_set,
                         shuffle=True,
                         batch_size=batch_size)

test_loader = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)

## Loss and optimizer


In [50]:
from torch import nn
from torch import optim

In [67]:
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

## Training the network

In [68]:
for epoch in range(num_epochs-3):
    for x_train, y_train in train_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
#         x_train = x_train/255
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        y_pred = model(x_train)
        
        loss = criterion(y_pred, y_train)
        
        loss.backward()
        
        optimizer.step()

    print(f" loss at the end of {epoch} epoch is {loss}")

print("Done Training")

 loss at the end of 0 epoch is 1.27711820602417
 loss at the end of 1 epoch is 1.1675723791122437
Done Training


In [69]:
def accuracy(test_loader, model,cuda=True):
    n_correct = 0
    n_total = 0
    with torch.no_grad():
        for x_test, y_test in test_loader:
            if cuda:
                x_test = x_test.to(device)
                y_test = y_test.to(device=device)

            preds = model(x_test)
            _, preds_idx = preds.max(1)
            n_correct += (preds_idx==y_test).sum().item()
            n_total += x_test.size(0)
    print(f"Out of {n_total} images {n_correct} where correctly classified")
    acc = (n_correct/n_total) * 100
    print(f"Accuracy of the model is {acc:.2f}")            

In [70]:
accuracy(test_loader, model)

Out of 10000 images 5868 where correctly classified
Accuracy of the model is 58.68


In [62]:
torch.cuda.empty_cache()